# Scikit-Learn: популярные модели и техника работы с ними

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor, SGDClassifier, Ridge, Lasso
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer, MinMaxScaler, RobustScaler, FunctionTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
RANDOM_STATE = 177013

In [ ]:
df = pd.read_excel('Concrete_Data.xls', sheet_name='Sheet1')

In [ ]:
df.describe().T

In [ ]:
df = df.drop_duplicates()

In [ ]:
df = df.rename(lambda x: x.split('(')[0].strip().replace(' ', '_').lower(), axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['concrete_compressive_strength'], axis=1),
                                                    df['concrete_compressive_strength'],
                                                    test_size=0.2,
                                                    random_state=177013,
                                                    shuffle=True,
                                                    )

In [ ]:
def fit_and_test(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)

    predictions = model.predict(X_train)
    mse = mean_squared_error (y_train, predictions)
    mae = mean_absolute_error(y_train, predictions)
    r2 = r2_score(y_train, predictions)
    
    print(f'Обучающая выборка - MSE: {mse:.2f}, MAE: {mae:.2f}, R2 {r2:.0%}.')
    
    predictions = model.predict(X_test)
    mse = mean_squared_error (y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    
    print(f'Тестовая выборка - MSE: {mse:.2f}, MAE: {mae:.2f}, R2 {r2:.0%}.')

## Фиктивные модели

In [ ]:
from sklearn.dummy import DummyClassifier, DummyRegressor

In [ ]:
for strategy in ['mean', 'median']:
    dummy = DummyRegressor(strategy=strategy)
    print(f'Стратегия: {strategy}')
    fit_and_test(dummy, X_train, y_train, X_test, y_test)

## Дерево решений

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

In [ ]:
model1 = DecisionTreeRegressor(random_state=RANDOM_STATE)
fit_and_test(model1, X_train, y_train, X_test, y_test)

In [ ]:
plt.figure(figsize=(12,6))
plot_tree(model1,feature_names=list(X_train.columns), filled=True);

In [ ]:
model2 = DecisionTreeRegressor(max_depth=2, random_state=RANDOM_STATE)
fit_and_test(model2, X_train, y_train, X_test, y_test)

In [ ]:
plt.figure(figsize=(12,6))
plot_tree(model2,feature_names=list(X_train.columns), filled=True);

## Проблема переобучения (overfit)

Оверфит возникает, когда модель учит обучающую выборку слишком тщательно, чтобы обобщить закономерности на дргуие данные. По сути, это значит, что сложность модели сравнима с объемом обучающей выборки. Из формулы MSE, к примеру, можно вывести:

$$
MSE_{train} = {\sigma^2}\left(1-\frac{d}{N}\right)
$$

$$
MSE_{test} = {\sigma^2}\left(1+\frac{d}{N}\right)
$$

Здесь $\sigma^2$ - дисперсия погрешности измерений, $d$ - мера сложности модели, $N$ - размер обучающей выборки ($N \geqslant d$).

Отсюда следует:

- данные с большой погрешностью трудно зафитить и трудно обобщить;
- большую выборку фитить труднее;
- но при этом большая выборка сделает модель точнее на тесте;
- сложную модель легче зафитить;
- но при этом сложной модели труднее обобщать.

Сложность модели определяется не только внутренней механикой, но и предобработкой данных.

## Кросс-валидация

Мы могли бы проверять метрику на тестовой выборке для разных гиперпараметров модели. Однако этим способом мы неминуемо придем к тому, что подстраиваемся под тест.

В качестве альтернативы можно было бы выделять еще одну промежуточную выборку, валидационную. Но тогда мы урезаем количество данных для обучения (либо в конце нам придется склеить выборки обратно и еще раз обучить модель).

Одним из решений является **кросс-валидация**: обучающая выборка разбивается на несколько равных частей, например, четыре. Из них модель обучается на трех, а на оставшейся оценивается. Затем процесс повторяется для всех четырех вариантов и метрика усредняется. Это позволяет неплохо оценить качество модели без влияния случайностей, возможных в отдельной выборке.

In [ ]:
model =  DecisionTreeRegressor(random_state=RANDOM_STATE)

In [ ]:
from sklearn.model_selection import cross_val_score, cross_validate

In [ ]:
cross_val_score(model, X_train, y_train, cv=4, scoring='neg_mean_squared_error')

In [ ]:
-cross_val_score(model, X_train, y_train, cv=4, scoring='neg_mean_squared_error').mean()

In [ ]:
scores = []
train_scores = []

for i in range(1, 21):
    model = DecisionTreeRegressor(max_depth=i, random_state=RANDOM_STATE)
    scores.append(cross_validate(model, X_train, y_train, cv=4, scoring='neg_mean_squared_error', return_train_score=True))
    

mse_train = [-x['train_score'].mean() for x in scores]    
mse_cv = [-x['test_score'].mean() for x in scores]
plt.plot(np.linspace(1, 20, 20), mse_train)
plt.plot(np.linspace(1, 20, 20), mse_cv)
plt.xticks(np.linspace(1, 20, 20))
plt.xlabel('Высота дерева')
plt.ylabel('MSE')
plt.legend(['На обучающей выборке', 'На кросс-валидации']);

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
model =  DecisionTreeRegressor(random_state=RANDOM_STATE)

In [ ]:
tree_params = {
                    'max_depth':range(1, 21),
                    'min_samples_split':range(2, 10),
                    'min_samples_leaf':range(1,10)
              }

In [ ]:
gcv = GridSearchCV(model, tree_params, cv=4, scoring='neg_mean_squared_error', n_jobs=-1)
gcv.fit(X_train, y_train)
print(f'Лучшие гиперпараметры: {dict(gcv.best_params_)}')
print(f'MSE: {(-gcv.best_score_):.2f}')
best_model = gcv.best_estimator_

In [ ]:
predictions = best_model.predict(X_test)
mse = mean_squared_error (y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Тестовая выборка - MSE: {mse:.2f}, MAE: {mae:.2f}, R2 {r2:.0%}.')

## Bias/variance tradeoff

Ошибку модели можно разложить на смещение (насколько среднее предсказание далеко от истины):

$$
y - \mathbb{E}[y_{pred}]
$$

и дисперсию (разброс диапазона предсказаний):

$$
\mathbb{E}[(y_{pred} - \mathbb{E}[y_{pred}])^2]
$$

Высокая дисперсия модели - показатель переобучения.

Давайте посмотрим, что влияет на эти показатели:

- с ростом обучающей выборки дисперсия снижается, смещение не меняется;
- с ростом сложности модели смещение снижается, дисперсия растет;
- добавление полезных признаков уменьшает смещение;
- удаление бесполезных признаков уменьшает дисперсию.

## Feature selection

Проводя исследовательский анализ, неплохо проверять связь с целевым признаком. Однако есть и методы, позволяющие оценить связь количественно или даже автоматизировать отбор.

### Embedded methods

Основной представитель встроенных методов - регуляризация, понижающая вес маловажных признаков. Мы знакомы с ней по линейным моделям.

### Wrapper methods

Эти методы обучают модели, добавляя и убавляя признаки, и сравнивают результаты (в том числе используя кросс-валидацию). В `sklearn.feature_selection` они представлены `SequentialFeatureSelector()` и `RFECV`.

Плюсы: весьма надежно.

Минусы: очень затратно.

### Filter methods

Оценивают признаки по некому критерию, позволяя нам выбирать по значимости. Есть много критериев значимости на разные случаи:

Между числовыми признаками: коэффициенты корреляции, F-тест (`sklearn.feature_selection.f_regression()`).

Между категориями: $\chi^2$-тест (`sklearn.feature_selection.chi2()`).

Между разнородными признаками: `scipy.stats.kendalltau()`, дисперсионный анализ, статистические тесты по группам.

Универсальным методом является взаимная информация: она работает и с числами, и с категориями, годится и для регрессии, и для классификации, выявляет любые зависимости, а не только линейные.

In [ ]:
from sklearn.feature_selection import mutual_info_regression

In [ ]:
mi_scores = mutual_info_regression(X_train, y_train, random_state=RANDOM_STATE)
mic_data = pd.DataFrame()
mic_data['feature'] = X_train.columns
mic_data['MIC'] = mi_scores
  
mic_data.sort_values(by='MIC', ascending=False)

### VIF

На примере линейных моделей мы видели, что сильно коррелирующие признаки раздувают дисперсию модели. Реальное влияние можно оценить с помощью **variance inflation factor**.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
vif_data = pd.DataFrame()
vif_data['feature'] = X_train.columns
vif_data['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(len(X_train.columns))]
  
vif_data.sort_values(by='VIF', ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(X_train.corr(), annot=True, cmap='icefire');

Как видим, здесь случай тяжелый. Эмпирически, VIF не должен превышать 10-15, но в данном случае нет признаков, зависящих чисто попарно, чтобы один из них выкинуть.

### Понижение размерности

#### Собственные значения и собственные вектора

Предположим, для квадратной матрицы $A$ можно подобрать такой ненулевой вектор $\bar u$, что:

$$
A\bar u = s \bar u
$$

где $s$ - скаляр.

Тогда $\bar u$ называется **собственным вектором (eigenvector)** матрицы $A$, а $s$ - **собственным значением (eigenvalue)** вектора $\bar u$.

У матрицы $N x N$ может быть N собственных векторов и N собственных значений. Иными словами, можно обобщить разложение в матричном виде:

$$
AU = \bar s U
$$

Для симметричной матрицы, такой как матрица ковариации, можно свести это к:

$$
A = U \bar s U^T
$$

Что же можно из этого вывести? Рассмотрим на двумерном примере в пространстве "вода - пластификатор":

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(df[['water', 'superplasticizer']])

In [ ]:
plt.scatter(X[:,0], X[:,1]);

In [ ]:
cov_matrix = np.cov(X, rowvar = False)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cov_matrix, annot=True, cmap='icefire');

In [ ]:
S, U = np.linalg.eigh(cov_matrix)
print("Собственные значения: \n", S)
print("Собственные вектора: \n", U)

**Собственные вектора** такого разложения создадут новый базис, на которой можно спроецировать наши точки:

In [ ]:
plt.scatter(X[:,0], X[:,1]);

for eigenvalue, eigenvector in zip(S, U.T):
    scaled_eigenvector = eigenvector * eigenvalue ** 0.5
    plt.arrow(X[:,0].mean(), X[:,1].mean(), scaled_eigenvector[0], scaled_eigenvector[1],
              color='r', width=0.01, head_width=0.2)

In [ ]:
plt.scatter(X @ U[:,0], X @ U[:,1]);

**Собственные значения** соответствуют объясненной дисперсии: чем выше значение, тем важнее соответствующий вектор:

In [ ]:
explained_variance = S / (len(X) - 1)
explained_variance / explained_variance.sum()

Как видим, один из компонентов объясняет более 80% дисперсии, эффективно заменяя два признака!

Этот способ понижения размерности называется **методом главных компонент (Principal Component Analysis).** В sklearn также есть соответствующая функция в нескольких вариантах.

#### PCA в sklearn

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

In [ ]:
plt.scatter(X_pca[:,0], X_pca[:,1]);

In [ ]:
pca.explained_variance_ratio_

In [ ]:
X_train_new = X_train.copy()

In [ ]:
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_train[['water', 'coarse_aggregate', 'fine_aggregate']])

In [ ]:
X_train_new[['pca1', 'pca2', 'pca3']] = X_pca
X_train_new = X_train_new.drop(['water', 'coarse_aggregate', 'fine_aggregate'], axis=1)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
vif_data = pd.DataFrame()
vif_data['feature'] = X_train_new.columns
vif_data['VIF'] = [variance_inflation_factor(X_train_new.values, i) for i in range(len(X_train_new.columns))]
  
vif_data.sort_values(by='VIF', ascending=False)

## Организуем конвейер

In [ ]:
columns_to_merge = ['water', 'coarse_aggregate', 'fine_aggregate']

**Деревянные модели довольно равнодушны к разнице в масштабе признаков (хотя иногда трансформация может помочь).**

In [ ]:
# Список скейлеров:
scaler_list = [
               None,
               #MinMaxScaler(),
               StandardScaler(),
               #RobustScaler(),
               PowerTransformer(),
               QuantileTransformer(random_state=RANDOM_STATE, n_quantiles=500),
               QuantileTransformer(random_state=RANDOM_STATE, output_distribution='normal', n_quantiles=500),
              ]

# Предобработка по группам признаков:
transformers = [
                   ("pca", PCA(n_components=2), columns_to_merge),
               ]

preprocessor = ColumnTransformer(transformers=transformers, remainder='passthrough', n_jobs=-1)

In [ ]:
def optimize(model, params, X, y, scoring='neg_mean_squared_error'):
    name = f'{type(model).__name__}'
    print(f'Оптимизация {name}...')
    pipe = Pipeline([
                        ('preprocessor', preprocessor),
                        ('scaler', None),                
                        ('model', model)
                    ])
    gcv = GridSearchCV(pipe, params, cv=4, scoring=scoring, n_jobs=-1)
    gcv.fit(X, y)
    print(f'Лучшие гиперпараметры: {dict(gcv.best_params_)}')
    print(f'MSE: {(-gcv.best_score_):.2f}')
    return gcv.best_estimator_

In [ ]:
tree_params = {
                    #'scaler':scaler_list,
                    'preprocessor__pca__n_components':[1, 2, 3],
                    'model__max_depth':range(1, 21),
                    'model__min_samples_split':range(2, 10),
                    'model__min_samples_leaf':range(1, 10)
              }

In [ ]:
best_tree = optimize(DecisionTreeRegressor(random_state=RANDOM_STATE), tree_params, X_train, y_train)

In [ ]:
predictions = best_tree.predict(X_test)
mse = mean_squared_error (y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Тестовая выборка - MSE: {mse:.2f}, MAE: {mae:.2f}, R2 {r2:.0%}.')

## Ансамбли деревянных моделей

In [ ]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

### Bagging (bootstrap aggregating)

Этот метод выбирает из обучающей выборки множество подвыборок (с возвращением), а также, в зависимости от реализации, может брать для каждой случайное подмножество признаков. На каждой строится дерево решений, и их результаты усредняются. Это неплохо снижает дисперсию модели:

In [ ]:
forest_params = {
                    'preprocessor__pca__n_components':[1, 2, 3],
                    'model__max_depth' : range(5, 25),
                    'model__n_estimators' : [200],
                    'model__min_samples_split':range(2, 5),
                }

In [ ]:
best_forest = optimize(RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1), forest_params, X_train, y_train)

In [ ]:
predictions = best_forest.predict(X_test)
mse = mean_squared_error (y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Тестовая выборка - MSE: {mse:.2f}, MAE: {mae:.2f}, R2 {r2:.0%}.')

### Pasting

Похожий метод, но выборки делаются без возвращения, а ветвление делается случайно, а не по критерию.

In [ ]:
best_et = optimize(ExtraTreesRegressor(random_state=RANDOM_STATE, n_jobs=-1), forest_params, X_train, y_train)

In [ ]:
predictions = best_et.predict(X_test)
mse = mean_squared_error (y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Тестовая выборка - MSE: {mse:.2f}, MAE: {mae:.2f}, R2 {r2:.0%}.')

### Boosting

В этом варианте деревья выращиваются последовательно, и каждое предсказывает ошибку предыдущего.

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

In [ ]:
boost_params = {
                    'preprocessor__pca__n_components':[1, 2, 3],
                    'model__max_depth' : range(1, 10),
                    'model__max_iter' : [200, 500, 1000],
                }

In [ ]:
best_hgb = optimize(HistGradientBoostingRegressor(random_state=RANDOM_STATE), boost_params, X_train, y_train)

In [ ]:
predictions = best_hgb.predict(X_test)
mse = mean_squared_error (y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Тестовая выборка - MSE: {mse:.2f}, MAE: {mae:.2f}, R2 {r2:.0%}.')

Популярные библиотеки градиентного бустинга:

- XGBoost
- Catboost
- LightGBM

## Ансамбли более высокого уровня

In [ ]:
from sklearn.ensemble import VotingRegressor, StackingRegressor

### Голосование

Усредняет предсказания разных моделей, позволяя снизить дисперсию еще сильнее.

In [ ]:
voters = [('et', best_et), ('gb', best_hgb)]
vote = VotingRegressor(estimators=voters, n_jobs=-1)

In [ ]:
vote.fit(X_train, y_train)

In [ ]:
predictions = vote.predict(X_test)
mse = mean_squared_error (y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Тестовая выборка - MSE: {mse:.2f}, MAE: {mae:.2f}, R2 {r2:.0%}.')

### Stacking/Blending

Похожие методы, которые используют предсказания отдельных моделей как признаки для метамодели. Как правило, метамодель берется линейная.

In [ ]:
stack = StackingRegressor(estimators=voters, final_estimator=LinearRegression(), cv=4, n_jobs=-1)

In [ ]:
stack.fit(X_train, y_train)

In [ ]:
predictions = stack.predict(X_test)
mse = mean_squared_error (y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Тестовая выборка - MSE: {mse:.2f}, MAE: {mae:.2f}, R2 {r2:.0%}.')

### Ансамбли классификации на коррекции ошибок

In [ ]:
from sklearn.multiclass import OutputCodeClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df_titanic = pd.read_csv('titanic.csv').drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)
df_titanic['Sex'] = (df_titanic['Sex'] == 'male').astype('int')

In [ ]:
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(df_titanic.drop(['Survived'], axis=1),
                                                    df_titanic['Survived'],
                                                    test_size=0.2,
                                                    random_state=177013,
                                                    shuffle=True,
                                                    stratify=df_titanic['Survived'],
                                                    )

In [ ]:
ecoc = OutputCodeClassifier(estimator=DecisionTreeClassifier(random_state=RANDOM_STATE), random_state=RANDOM_STATE, n_jobs=-1)

In [ ]:
ecoc_params = {
                    #'scaler':scaler_list,
                    'preprocessor':[None],
                    'model__estimator__max_depth':range(1, 21),
                    'model__estimator__min_samples_split':range(2, 10),
                    'model__estimator__min_samples_leaf':range(1, 10)
              }

In [ ]:
best_ecoc = optimize(ecoc, ecoc_params, X_train_t, y_train_t, scoring="balanced_accuracy")

In [ ]:
accuracy_score(y_test_t, best_ecoc.predict(X_test_t))

## Другие популярные модели

### Метод ближайших соседей

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

**Метод ближайших соседей чувствителен к масштабу. Вам понадобится скейлер.**

In [ ]:
knn_params = {
                    'scaler':scaler_list,
                    'preprocessor__pca__n_components':[1, 2, 3],
                    'model__n_neighbors' : range(1, 10),
                    'model__weights' : ['uniform', 'distance'],
                    'model__metric' :['minkowski', 'cityblock', 'cosine']
    
             }

In [ ]:
best_knn = optimize(KNeighborsRegressor(n_jobs=-1), knn_params, X_train, y_train)

In [ ]:
predictions = best_knn.predict(X_test)
mse = mean_squared_error (y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Тестовая выборка - MSE: {mse:.2f}, MAE: {mae:.2f}, R2 {r2:.0%}.')

Этот метод нетрудно адаптировать для заполнения пропусков в данных:

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
imputer = KNNImputer(n_neighbors=5, weights='uniform', add_indicator=True).set_output(transform='pandas')

In [ ]:
df_titanic = pd.read_csv('titanic.csv').drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)
df_titanic['Sex'] = (df_titanic['Sex'] == 'male').astype('int')

In [ ]:
df_titanic = imputer.fit_transform(df_titanic)

In [ ]:
df_titanic.tail()

### Метод опорных векторов (SVM)

In [ ]:
from sklearn.svm import SVR

**Метод опорных векторов также чувствителен к масштабу.**

In [ ]:
svr_params = {
                    'scaler':scaler_list,
                    'preprocessor__pca__n_components':[1, 2, 3],
                    'model__epsilon': [0.1, 0.2, 0.3, 0.4, 0.5],
                    'model__C':[0.1, 1, 10, 100, 1000, 10000]
             }

In [ ]:
best_svr = optimize(SVR(), svr_params, X_train, y_train)

In [ ]:
predictions = best_svr.predict(X_test)
mse = mean_squared_error (y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Тестовая выборка - MSE: {mse:.2f}, MAE: {mae:.2f}, R2 {r2:.0%}.')

### GLM

In [ ]:
from sklearn.linear_model import GammaRegressor, TweedieRegressor, PoissonRegressor

Обобщение линейных моделей для разных распределений целевой переменной. Например:

- Нормальное распределение (`power=0`). Сводится к обычной линейной регрессии.
- Распределение Пуассона (`power=1` или `PoissonRegressor()`). Хорошо подходит, если надо предсказать, например, штучное количество.
- Гамма-распределение(`power=2` или `GammaRegressor()`). Обобщение экспоненциального. Подходит, например, для затраченного времени или расходов на событие.
- Обратное нормальному распределение(`power=2`). Для целевых переменных с большим "хвостом".
- Гибрид гамма-Пуассона(`power=1.5`). Например, сумма расходов за год.

In [ ]:
tw_params = {
                    'scaler':scaler_list,
                    'preprocessor__pca__n_components':[1, 2, 3],
                    'model__power':np.linspace(0, 4, 5),
                    'model__alpha':np.logspace(-6, 2, 9),
               }

In [ ]:
best_tw = optimize(TweedieRegressor(), tw_params, X_train, y_train)

In [ ]:
predictions = best_tw.predict(X_test)
mse = mean_squared_error (y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Тестовая выборка - MSE: {mse:.2f}, MAE: {mae:.2f}, R2 {r2:.0%}.')

### Кригинг

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor

In [ ]:
krig_params = {
                    'scaler':scaler_list,
                    'preprocessor__pca__n_components':[1, 2, 3],
                    #'model__alpha': np.logspace(-10, 1, 12),
                    'model__normalize_y' :[True, False],
    
             }

In [ ]:
best_krig = optimize(GaussianProcessRegressor(random_state=RANDOM_STATE, alpha=0.1), krig_params, X_train, y_train)

In [ ]:
predictions = best_krig.predict(X_test)
mse = mean_squared_error (y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Тестовая выборка - MSE: {mse:.2f}, MAE: {mae:.2f}, R2 {r2:.0%}.')

In [ ]:
for n, (mean, std) in enumerate(zip(*best_krig.predict(X_test, return_std=True))):
    print(f'{mean} ± {std * 1.96} vs {y_test.iloc[n]}')
    if n >=10:
        break

In [ ]:
from sklearn.metrics import make_scorer

In [ ]:
def interval_hit(estimator, X, y):
    mean, std = estimator.predict(X, return_std=True)
    lower_bound = mean - std * 1.96
    upper_bound = mean + std * 1.96
    return np.mean((lower_bound <= y) & (y <= upper_bound))

In [ ]:
def optimize_for_hit(model, params, X, y):
    name = f'{type(model).__name__}'
    print(f'Оптимизация {name}...')
    pipe = Pipeline([
                        ('preprocessor', preprocessor),
                        ('scaler', None),                
                        ('model', model)
                    ])
    gcv = GridSearchCV(pipe, params, cv=4, scoring=interval_hit, n_jobs=-1)
    gcv.fit(X, y)
    print(f'Лучшие гиперпараметры: {dict(gcv.best_params_)}')
    print(f'Попаданий в интервал: {(gcv.best_score_):.2%}')
    return gcv.best_estimator_

In [ ]:
krig_params = {
                    'scaler':scaler_list,
                    'preprocessor__pca__n_components':[1, 2, 3],
                    'model__alpha': np.logspace(-10, 2, 13),
                    'model__normalize_y' :[True, False],
    
             }

In [ ]:
best_krig = optimize_for_hit(GaussianProcessRegressor(random_state=RANDOM_STATE), krig_params, X_train, y_train)

In [ ]:
predictions = best_krig.predict(X_test)
mse = mean_squared_error (y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Тестовая выборка - MSE: {mse:.2f}, MAE: {mae:.2f}, R2 {r2:.0%}.')

In [ ]:
interval_hit(best_krig, X_test, y_test)

In [ ]:
for n, (mean, std) in enumerate(zip(*best_krig.predict(X_test, return_std=True))):
    print(f'{mean} ± {std * 1.96} vs {y_test.iloc[n]}')
    if n >=10:
        break

### Наивный Байес

In [ ]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB

In [ ]:
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(df_titanic.drop(['Survived'], axis=1),
                                                    df_titanic['Survived'],
                                                    test_size=0.2,
                                                    random_state=177013,
                                                    shuffle=True,
                                                    stratify=df_titanic['Survived'],
                                                    )

In [ ]:
nb = GaussianNB().fit(X_train_t, y_train_t)

In [ ]:
accuracy_score(y_test_t, nb.predict(X_test_t))

## Feature importance

К определению итоговой важности признаков есть несколько подходов:

1. Признаки, которые модель чаще использовала для обобщения при учебе. У многих классов моделей есть свойство `feature_importances_`.

In [ ]:
mdi_importances = pd.Series(best_forest['model'].feature_importances_, index=best_forest[:-1].get_feature_names_out())

In [ ]:
mdi_importances.plot(kind='bar');
plt.xlabel('Признак')
plt.ylabel('Влияние');

Недостаток подхода в том, что признаки с большой кардинальностью получают больший вес. Кроме того, понижение размерности не позволяет судить об изначальном влиянии.

2. Признаки, которые сильнее всего влияют на метрику.

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
perm_importances = pd.Series(permutation_importance(best_forest, X_test, y_test, random_state=RANDOM_STATE, scoring='r2', n_jobs=-1)['importances_mean'], index=X_test.columns)

In [ ]:
perm_importances.plot(kind='bar');
plt.xlabel('Признак')
plt.ylabel('Влияние');

2. Признаки, которые сильнее всего влияют на предсказание (не обязательно корректное).

In [ ]:
np.bool = bool

In [ ]:
import shap

explainer = shap.Explainer(best_forest.named_steps['model'])
X = best_forest.named_steps['preprocessor'].transform(X_test)
shap_values = explainer(X)

# Plot the summary plot of SHAP values
shap.summary_plot(shap_values, X, feature_names=X_test.columns, plot_type='bar')

## Сохранение и загрузка обученных моделей

Через joblib (рекомендуется):

In [ ]:
from joblib import dump, load
dump(nb, 'nb.joblib') 

In [ ]:
clf = load('nb.joblib') 

In [ ]:
clf.predict([[2.0, 1.0, 27.0, 0.0, 0.0, 13.00, 0.0]])

Через pickle:

In [ ]:
import pickle
s = pickle.dumps(clf)

In [ ]:
with open('nb.pickle', 'wb') as file:
    pickle.dump(nb, file)

In [ ]:
with open('nb.pickle', 'rb') as file:
    clf2 = pickle.load(file)

In [ ]:
clf2.predict([[2.0, 1.0, 27.0, 0.0, 0.0, 13.00, 0.0]])

# Домашненее задание

## Easy

In [ ]:
from sklearn.feature_selection import mutual_info_classif

In [ ]:
df_titanic = pd.read_csv('titanic.csv')

Оцените взаимную информацию, VIF и корреляции в датасете "Титаник". Для этого нужно сперва удалить ненужные столбцы, а оставшиеся категории превратить в числа.

Обратите внимание, что это задача классификации на целевой признак `Survived`, а значит, вам нужен `mutual_info_classif()`, а не `mutual_info_regressor()`.

In [ ]:
# Ваш код ниже:


## Normal/Hard

Вам предстоит построить конвейер sklearn, проводящий классификацию в датасете "Титаник". Если вы не сможете выполнить все условия, обойдите их более простыми способами (например, удаляйте пропуски, а не заполняйте; если вам не удается подобрать гиперпараметры, обучите конвейер с параметрами модели по умолчанию и т. д.). Наивысшую оценку получат работы, выполнившие все пункты.

Для начала выделите тестовую и обучающую выборки.

Конвейер должен проводить следующие действия:

- выбросить ненужные столбцы;
- заполнить пропуски с помощью `KNNImputer()`;
- отмасштабировать данные, если это нужно для модели;
- содержать на выходе модель классификации по вашему выбору.

Подберите наилучшие параметры конвейера (в идеале не только модели, но и, например, импутера) с помощью `GridSearchCV` или `RandomizedSearchCV`.

Проверьте результат на тестовой выборке. Сделайте вывод.

Сохраните конвейер в файл и загрузите его вместе с тетрадью.

In [ ]:
# Ваш код ниже:
